# Deep Support Vector Data Description with Variational Autoencoder in Pytorch

## Imports

In [1]:
# This packages should be used with lower versions because of incompatibility issues
!pip install "click>=7,<8"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.5 requires click>=8.0, but you have click 7.1.2 which is incompatible.
dask 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
distributed 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
fiona 1.9.6 requires click~=8.0, but you have click 7.1.2 which is incompatible.
pip-tools 6.13.0 requires click>=8, but you have click 7.1.2 which is incompatible.


In [2]:
# Base Python Imports
import json
import time
import click
import random
import logging
from abc import ABC, abstractmethod

# Library Imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import roc_auc_score
from torchvision.utils import make_grid
from tqdm.notebook import tqdm, trange

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10, MNIST
import torchvision.transforms as transforms

## `base` folder

In [3]:
import logging
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader
from abc import ABC, abstractmethod

### Contents of `base_dataset.py`

In [4]:
class BaseADDataset(ABC):
    """Anomaly detection dataset base class."""

    def __init__(self, root: str):
        super().__init__()
        self.root = root  # root path to data

        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = None  # tuple with original class labels that define the normal class
        self.outlier_classes = None  # tuple with original class labels that define the outlier class

        self.train_set = None  # must be of type torch.utils.data.Dataset
        self.test_set = None  # must be of type torch.utils.data.Dataset

    @abstractmethod
    def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0) -> (
            DataLoader, DataLoader):
        """Implement data loaders of type torch.utils.data.DataLoader for train_set and test_set."""
        pass

    def __repr__(self):
        return self.__class__.__name__

### Contents of `base_net.py`

In [5]:
class BaseNet(nn.Module):
    """Base class for all neural networks."""

    def __init__(self):
        super().__init__()
        self.logger = logging.getLogger(self.__class__.__name__)
        self.rep_dim = None  # representation dimensionality, i.e. dim of the last layer

    def forward(self, *input):
        """
        Forward pass logic
        :return: Network output
        """
        raise NotImplementedError

    def summary(self):
        """Network summary."""
        net_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in net_parameters])
        self.logger.info('Trainable parameters: {}'.format(params))
        self.logger.info(self)

### Contents of `base_trainer.py`

In [6]:
class BaseTrainer(ABC):
    """Trainer base class."""

    def __init__(self,
                 optimizer_name: str,
                 lr: float,
                 n_epochs: int,
                 lr_milestones: tuple,
                 batch_size: int,
                 weight_decay: float,
                 device: str,
                 n_jobs_dataloader: int):
        super().__init__()
        self.optimizer_name = optimizer_name
        self.lr = lr
        self.n_epochs = n_epochs
        self.lr_milestones = lr_milestones
        self.batch_size = batch_size
        self.weight_decay = weight_decay
        self.device = device
        self.n_jobs_dataloader = n_jobs_dataloader

    @abstractmethod
    def train(self, dataset: BaseADDataset, net: BaseNet) -> BaseNet:
        """
        Implement train method that trains the given network using the train_set of dataset.
        :return: Trained net
        """
        pass

    @abstractmethod
    def test(self, dataset: BaseADDataset, net: BaseNet):
        """
        Implement test method that evaluates the test_set of dataset on the given network.
        """
        pass

### Contents of `torchvision_dataset.py`

In [7]:
class TorchvisionDataset(BaseADDataset):
    """TorchvisionDataset class for datasets already implemented in torchvision.datasets."""

    def __init__(self, root: str):
        super().__init__(root)

    def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0) -> (
            DataLoader, DataLoader):
        train_loader = DataLoader(
            dataset=self.train_set,
            batch_size=batch_size,
            shuffle=shuffle_train,
            num_workers=num_workers
        )
        test_loader = DataLoader(
            dataset=self.test_set,
            batch_size=batch_size,
            shuffle=shuffle_test,
            num_workers=num_workers
        )
        return train_loader, test_loader

## `utils` folder

### Contents of `config.py`

In [8]:
class Config(object):
    """Base class for experimental setting/configuration."""

    def __init__(self, settings):
        self.settings = settings

    def load_config(self, import_json):
        """Load settings dict from import_json (path/filename.json) JSON-file."""

        with open(import_json, 'r') as fp:
            settings = json.load(fp)

        for key, value in settings.items():
            self.settings[key] = value

    def save_config(self, export_json):
        """Save settings dict to export_json (path/filename.json) JSON-file."""

        with open(export_json, 'w') as fp:
            json.dump(self.settings, fp)

### Contents of `plot.py`

In [9]:
def plot_images_grid(x: torch.tensor, export_img, title: str = '', nrow=8, padding=2, normalize=False, pad_value=0):
    """Plot 4D Tensor of images of shape (B x C x H x W) as a grid."""

    grid = make_grid(x, nrow=nrow, padding=padding, normalize=normalize, pad_value=pad_value)
    npgrid = grid.cpu().numpy()

    plt.imshow(np.transpose(npgrid, (1, 2, 0)), interpolation='nearest')

    ax = plt.gca()
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    if not (title == ''):
        plt.title(title)

    plt.savefig(export_img, bbox_inches='tight', pad_inches=0.1)

## `datasets` folder

In [10]:
# Tranformation class that ensures the data is in range [0, 1], with no 0-eps, 1+eps values
class ClampImageTo01():
    """Clamp image pixel values to be between 0 and 1."""

    def __call__(self, img):
        return torch.clamp(img, 0, 1)

### Contents of `cifar10.py`

In [11]:
class CIFAR10_Dataset(TorchvisionDataset):

    def __init__(self, root: str, normal_class=5):
        super().__init__(root)

        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = tuple([normal_class])
        self.outlier_classes = list(range(0, 10))
        self.outlier_classes.remove(normal_class)

        # Pre-computed min and max values (after applying GCN) from train data per class
        min_max = [(-28.94083453598571, 13.802961825439636),
                   (-6.681770233365245, 9.158067708230273),
                   (-34.924463588638204, 14.419298165027628),
                   (-10.599172931391799, 11.093187820377565),
                   (-11.945022995801637, 10.628045447867583),
                   (-9.691969487694928, 8.948326776180823),
                   (-9.174940012342555, 13.847014686472365),
                   (-6.876682005899029, 12.282371383343161),
                   (-15.603507135507172, 15.2464923804279),
                   (-6.132882973622672, 8.046098172351265)]

        # CIFAR-10 preprocessing: GCN (with L1 norm) and min-max feature scaling to [0,1]
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
                                        transforms.Normalize([min_max[normal_class][0]] * 3,
                                                             [min_max[normal_class][1] - min_max[normal_class][0]] * 3),
                                        ClampImageTo01()
                                        ])

        target_transform = transforms.Lambda(lambda x: int(x in self.outlier_classes))

        train_set = MyCIFAR10(root=self.root, train=True, download=True,
                              transform=transform, target_transform=target_transform)
        # Subset train set to normal class
        train_idx_normal = get_target_label_idx(train_set.train_labels, self.normal_classes)
        self.train_set = Subset(train_set, train_idx_normal)

        self.test_set = MyCIFAR10(root=self.root, train=False, download=True,
                                  transform=transform, target_transform=target_transform)


class MyCIFAR10(CIFAR10):
    """Torchvision CIFAR10 class with patch of __getitem__ method to also return the index of a data sample."""

    def __init__(self, *args, **kwargs):
        super(MyCIFAR10, self).__init__(*args, **kwargs)

    def __getitem__(self, index):
        """Override the original method of the CIFAR10 class.
        Args:
            index (int): Index
        Returns:
            triple: (image, target, index) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index  # only line changed

### Contents of `mnist.py`

In [57]:
class MNIST_Dataset(TorchvisionDataset):

    def __init__(self, root: str, normal_class=0):
        super().__init__(root)

        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = tuple([normal_class])
        self.outlier_classes = list(range(0, 10))
        self.outlier_classes.remove(normal_class)

        # Pre-computed min and max values (after applying GCN) from train data per class
        min_max = [(-0.8826567065619495, 9.001545489292527),
                   (-0.6661464580883915, 20.108062262467364),
                   (-0.7820454743183202, 11.665100841080346),
                   (-0.7645772083211267, 12.895051191467457),
                   (-0.7253923114302238, 12.683235701611533),
                   (-0.7698501867861425, 13.103278415430502),
                   (-0.778418217980696, 10.457837397569108),
                   (-0.7129780970522351, 12.057777597673047),
                   (-0.8280402650205075, 10.581538445782988),
                   (-0.7369959242164307, 10.697039838804978)]

        # MNIST preprocessing: GCN (with L1 norm) and min-max feature scaling to [0,1]
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
                                        transforms.Normalize([min_max[normal_class][0]],
                                                             [min_max[normal_class][1] - min_max[normal_class][0]]),
                                        ClampImageTo01(),
                                        # TODO: remove this sometimes
                                        # transforms.Lambda(torch.flatten)
                                        ])

        target_transform = transforms.Lambda(lambda x: int(x in self.outlier_classes))

        train_set = MyMNIST(root=self.root, train=True, download=True,
                            transform=transform, target_transform=target_transform)
        # Subset train_set to normal class
        train_idx_normal = get_target_label_idx(train_set.train_labels.clone().data.cpu().numpy(), self.normal_classes)
        self.train_set = Subset(train_set, train_idx_normal)

        self.test_set = MyMNIST(root=self.root, train=False, download=True,
                                transform=transform, target_transform=target_transform)


class MyMNIST(MNIST):
    """Torchvision MNIST class with patch of __getitem__ method to also return the index of a data sample."""

    def __init__(self, *args, **kwargs):
        super(MyMNIST, self).__init__(*args, **kwargs)

    def __getitem__(self, index):
        """Override the original method of the MNIST class.
        Args:
            index (int): Index
        Returns:
            triple: (image, target, index) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index  # only line changed

### Contents of `preprocessing.py`

In [13]:
def get_target_label_idx(labels, targets):
    """
    Get the indices of labels that are included in targets.
    :param labels: array of labels
    :param targets: list/tuple of target labels
    :return: list with indices of target labels
    """
    return np.argwhere(np.isin(labels, targets)).flatten().tolist()


def global_contrast_normalization(x: torch.tensor, scale='l2'):
    """
    Apply global contrast normalization to tensor, i.e. subtract mean across features (pixels) and normalize by scale,
    which is either the standard deviation, L1- or L2-norm across features (pixels).
    Note this is a *per sample* normalization globally across features (and not across the dataset).
    """

    assert scale in ('l1', 'l2')

    n_features = int(np.prod(x.shape))

    mean = torch.mean(x)  # mean over all features (pixels) per sample
    x -= mean

    if scale == 'l1':
        x_scale = torch.mean(torch.abs(x))

    if scale == 'l2':
        x_scale = torch.sqrt(torch.sum(x ** 2)) / n_features

    x /= x_scale

    return x

### Contents of `main.py`

In [14]:
def load_dataset(dataset_name, data_path, normal_class):
    """Loads the dataset."""

    implemented_datasets = ('mnist', 'cifar10')
    assert dataset_name in implemented_datasets

    dataset = None

    if dataset_name == 'mnist':
        dataset = MNIST_Dataset(root=data_path, normal_class=normal_class)

    if dataset_name == 'cifar10':
        dataset = CIFAR10_Dataset(root=data_path, normal_class=normal_class)

    return dataset

## `networks` folder

### Contents of `cifar10_LeNet_elu.py`

In [15]:
class CIFAR10_LeNet_ELU(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 128
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, bias=False, padding=2)
        self.bn2d1 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, bias=False, padding=2)
        self.bn2d2 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, bias=False, padding=2)
        self.bn2d3 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=self.rep_dim, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.elu(self.bn2d1(x)))
        x = self.conv2(x)
        x = self.pool(F.elu(self.bn2d2(x)))
        x = self.conv3(x)
        x = self.pool(F.elu(self.bn2d3(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


class CIFAR10_LeNet_ELU_Autoencoder(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 128
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Encoder (must match the Deep SVDD network above)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv1.weight)
        self.bn2d1 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv2.weight)
        self.bn2d2 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv3.weight)
        self.bn2d3 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=self.rep_dim, bias=False)
        self.bn1d = nn.BatchNorm1d(num_features=self.rep_dim, eps=1e-04, affine=False)

        # Decoder
        self.deconv1 = nn.ConvTranspose2d(in_channels=int(self.rep_dim / (4 * 4)), out_channels=128, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv1.weight)
        self.bn2d4 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv2.weight)
        self.bn2d5 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv3.weight)
        self.bn2d6 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv4.weight)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.elu(self.bn2d1(x)))
        x = self.conv2(x)
        x = self.pool(F.elu(self.bn2d2(x)))
        x = self.conv3(x)
        x = self.pool(F.elu(self.bn2d3(x)))
        x = x.view(x.size(0), -1)
        x = self.bn1d(self.fc1(x))
        x = x.view(x.size(0), int(self.rep_dim / (4 * 4)), 4, 4)
        x = F.elu(x)
        x = self.deconv1(x)
        x = F.interpolate(F.elu(self.bn2d4(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.elu(self.bn2d5(x)), scale_factor=2)
        x = self.deconv3(x)
        x = F.interpolate(F.elu(self.bn2d6(x)), scale_factor=2)
        x = self.deconv4(x)
        x = torch.sigmoid(x)
        return x

### Contents of `cifar10_LeNet.py`

In [16]:
class CIFAR10_LeNet(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 128
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, bias=False, padding=2)
        self.bn2d1 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, bias=False, padding=2)
        self.bn2d2 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, bias=False, padding=2)
        self.bn2d3 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=self.rep_dim, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn2d1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2d2(x)))
        x = self.conv3(x)
        x = self.pool(F.leaky_relu(self.bn2d3(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


class CIFAR10_LeNet_Autoencoder(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 128
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Encoder (must match the Deep SVDD network above)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv1.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d1 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv2.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d2 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.conv3.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d3 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=self.rep_dim, bias=False)
        self.bn1d = nn.BatchNorm1d(num_features=self.rep_dim, eps=1e-04, affine=False)

        # Decoder
        self.deconv1 = nn.ConvTranspose2d(in_channels=int(self.rep_dim / (4 * 4)), out_channels=128, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv1.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d4 = nn.BatchNorm2d(num_features=128, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv2.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d5 = nn.BatchNorm2d(num_features=64, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv3.weight, gain=nn.init.calculate_gain('leaky_relu'))
        self.bn2d6 = nn.BatchNorm2d(num_features=32, eps=1e-04, affine=False)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=5, bias=False, padding=2)
        nn.init.xavier_uniform_(self.deconv4.weight, gain=nn.init.calculate_gain('leaky_relu'))

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn2d1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2d2(x)))
        x = self.conv3(x)
        x = self.pool(F.leaky_relu(self.bn2d3(x)))
        x = x.view(x.size(0), -1)
        x = self.bn1d(self.fc1(x))
        x = x.view(x.size(0), int(self.rep_dim / (4 * 4)), 4, 4)
        x = F.leaky_relu(x)
        x = self.deconv1(x)
        x = F.interpolate(F.leaky_relu(self.bn2d4(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.leaky_relu(self.bn2d5(x)), scale_factor=2)
        x = self.deconv3(x)
        x = F.interpolate(F.leaky_relu(self.bn2d6(x)), scale_factor=2)
        x = self.deconv4(x)
        x = torch.sigmoid(x)
        return x

### Contents of `mnist_LeNet.py`

In [17]:
# Acknowledgements to pytorch-generative module

# Utils for VAE
# @torch.jit.script
def variance(log_stdev):
    """Returns the variance given the logarithm of standard deviation."""
    return log_stdev.exp().pow(2)


# @torch.jit.script
def kl_div_unit_gaussian(mean, log_stdev):
    """
    Returns `KL(p || N(0, 1))` where `p` is a Gaussian with diagonal covariance.
    Source: https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
    """
    return -0.5 * (1 + 2 * log_stdev - mean.pow(2) - variance(log_stdev))

# @torch.jit.script
def kl_div_gaussian(p_mean, p_log_stdev, q_mean, q_log_stdev):
    """
    Returns `KL(p || q)` where `p` and `q` are Gaussians with diagonal covariance.
    Source: https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
    """
    mean_delta = (p_mean - q_mean) ** 2
    log_std_delta = q_log_stdev - p_log_stdev
    p_var = variance(p_log_stdev)
    q_var = 2 * variance(q_log_stdev)
    return -0.5 + log_std_delta + (p_var + mean_delta) / q_var

# @torch.jit.script
def sample_from_gaussian(mean, log_stdev):
    """
    Returns a sample from a Gaussian with diagonal covariance.
    """
    return mean + log_stdev.exp() * torch.randn_like(mean)

In [18]:
from torch.nn import init
from torch.autograd import Variable


# Acknowledgements: https://github.com/wohlert/semi-supervised-pytorch
class LatentGaussianSample(nn.Module):
    """
    Layer that represents a sample from a Gaussian distribution.
    """
    def __init__(self, in_features, out_features):
        super(GaussianSample, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.mean_fc = nn.Linear(in_features, out_features)
        self.log_stdev_fc = nn.Linear(in_features, out_features)

    def reparametrize(self, mean, log_stdev):
        eps = Variable(torch.randn_like(mean), requires_grad=False)
        return mean + log_stdev.exp() * eps

    def forward(self, x):
        mean = self.mean_fc(x)
        log_stdev = F.softplus(self.log_stdev_fc(x))
        return self.reparametrize(mean, log_stdev), mean, log_stdev


class MNIST_LeNet_Encoder(BaseNet):
    def __init__(self, rep_dim=32, bias=True):
        super().__init__()

        self.rep_dim = rep_dim
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, bias=bias, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=8, eps=1e-04, affine=False)

        self.conv2 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=5, bias=bias, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=4, eps=1e-04, affine=False)

        # We will need an output that is two times larger than the latent space (one time for mean values, one time for stdev values)
        # self.mean_fc = nn.Linear(in_features=4 * 7 * 7, out_features=self.rep_dim, bias=bias)
        # self.log_stdev_fc = nn.Linear(in_features=4 * 7 * 7, out_features=self.rep_dim, bias=bias)
        self.latent_net = LatentGaussianSample(in_features=4 * 7 * 7, out_features=self.rep_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        z, mean, log_stdev = self.latent_net(x)
        return z, mean, log_stdev

class MNIST_LeNet_Decoder(BaseNet):
    def __init__(self, rep_dim=32, bias=True):
        super().__init__()

        self.rep_dim = rep_dim

        self.deconv1 = nn.ConvTranspose2d(in_channels=2, out_channels=4, kernel_size=5, bias=bias, padding=2)
        self.bn3 = nn.BatchNorm2d(num_features=4, eps=1e-04, affine=False)

        self.deconv2 = nn.ConvTranspose2d(in_channels=4, out_channels=8, kernel_size=5, bias=bias, padding=3)
        self.bn4 = nn.BatchNorm2d(num_features=8, eps=1e-04, affine=False)

        self.deconv3 = nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=5, bias=bias, padding=2)

    def forward(self, x):
        x = x.view(x.size(0), int(self.rep_dim / 16), 4, 4)
        x = F.interpolate(F.leaky_relu(x), scale_factor=2)
        x = self.deconv1(x)
        x = F.interpolate(F.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.deconv3(x)
        x = torch.sigmoid(x)
        return x


class MNIST_LeNet_VAE(BaseNet):
    def __init__(self, rep_dim=32, bias=True):
        super().__init__()

        self.rep_dim = rep_dim
        # Todo: make it self
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.encoder = MNIST_LeNet_Encoder(rep_dim, bias)
        self.decoder = MNIST_LeNet_Decoder(rep_dim, bias)

        # Init linear layers
        for i, m in enumerate(self.modules()):
            # print(i, m)
            if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.zeros_(m.bias)

    def forward(self, x):
        # Encode the initial data
        z, mean, log_stdev = self.encoder(x)

        # Compute mean and log_std
        # First half of the outputs are going to be means, the second half is going to be natural logarith of stdevs

        # mean, log_stdev = torch.split(encoded, split_size_or_sections=self.rep_dim, dim=1)
        # TODO: try with mean here
        kl_divs = kl_div_unit_gaussian(mean, log_stdev).sum(dim=1)
        # print(f"Shapes of VAE: x: {x.shape}, enc: {encoded.shape}, kl_divs: {kl_divs.shape}, mean: {mean.shape}, log_stdev: {log_stdev.shape}")

        # Compute z, representing latent features
        # z = sample_from_gaussian(mean, log_stdev)

        # Pass z to decoder and try to compute initial data
        decoded = self.decoder(z)

        return decoded, z, kl_divs

    def sample(self, num_samples=1):
        # Generate a random sample of latent space and decode it
        latent_size = self.rep_dim
        shape = (num_samples, latent_size)
        z = torch.randn(shape, device=self.device)
        return self.decoder(z)

### Contents of `main.py`

In [19]:
def build_network(net_name):
    """Builds the neural network."""

    implemented_networks = ('mnist_LeNet', 'cifar10_LeNet', 'cifar10_LeNet_ELU')
    assert net_name in implemented_networks

    net = None

    if net_name == 'mnist_LeNet':
        net = MNIST_LeNet_VAE()

    if net_name == 'cifar10_LeNet':
        net = CIFAR10_LeNet()

    if net_name == 'cifar10_LeNet_ELU':
        net = CIFAR10_LeNet_ELU()

    return net


def build_autoencoder(net_name):
    """Builds the corresponding autoencoder network."""

    implemented_networks = ('mnist_LeNet', 'cifar10_LeNet', 'cifar10_LeNet_ELU')
    assert net_name in implemented_networks

    ae_net = None

    if net_name == 'mnist_LeNet':
        ae_net = MNIST_LeNet_Autoencoder()

    if net_name == 'cifar10_LeNet':
        ae_net = CIFAR10_LeNet_Autoencoder()

    if net_name == 'cifar10_LeNet_ELU':
        ae_net = CIFAR10_LeNet_ELU_Autoencoder()

    return ae_net

## `optim` folder

In [20]:
def MSELoss(x, x_hat):
    """
    Computes the mean squared error loss for inputs and outputs
    """
    scores = torch.sum((x_hat - x) ** 2, dim=tuple(range(1, x_hat.dim())))
    loss = torch.mean(scores)
    return loss

### Contents of `ae_trainer.py`

In [21]:
class AETrainer(BaseTrainer):

    def __init__(self,
                 optimizer_name: str = 'adam',
                 lr: float = 0.001,
                 n_epochs: int = 150,
                 lr_milestones: tuple = (),
                 batch_size: int = 128,
                 weight_decay: float = 1e-6,
                 device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        super().__init__(optimizer_name, lr, n_epochs, lr_milestones, batch_size, weight_decay, device,
                         n_jobs_dataloader)

    def objective(self, x, x_hat):
        loss = MSELoss(x, x_hat)
        return loss

    def test_score(self, x, x_hat):
        scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
        return scores

    def train(self, dataset: BaseADDataset, ae_net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        ae_net = ae_net.to(self.device)

        # Get train data loader
        train_loader, _ = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Set optimizer (Adam optimizer for now)
        optimizer = optim.Adam(ae_net.parameters(), lr=self.lr, weight_decay=self.weight_decay,
                               amsgrad=self.optimizer_name == 'amsgrad')

        # Set learning rate scheduler
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_milestones, gamma=0.1)

        # Training
        logger.info('Starting pretraining...')
        start_time = time.time()
        ae_net.train()
        for epoch in trange(self.n_epochs, desc="Epoch"):

            scheduler.step()
            if epoch in self.lr_milestones:
                logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

            loss_epoch = 0.0
            n_batches = 0
            epoch_start_time = time.time()
            for data in tqdm(train_loader):
                inputs, _, _ = data
                inputs = inputs.to(self.device)

                # Zero the network parameter gradients
                optimizer.zero_grad()

                # Update network parameters via backpropagation: forward + backward + optimize
                # For autoencoder we are using MSE Loss
                # TODO: Try with nn.MSELoss()
                outputs = ae_net(inputs)
                loss = self.objective(inputs, outputs)
                loss.backward()
                optimizer.step()

                loss_epoch += loss.item()
                n_batches += 1

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                        .format(epoch + 1, self.n_epochs, epoch_train_time, loss_epoch / n_batches))

        pretrain_time = time.time() - start_time
        logger.info('Pretraining time: %.3f' % pretrain_time)
        logger.info('Finished pretraining.')

        return ae_net

    def test(self, dataset: BaseADDataset, ae_net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        ae_net = ae_net.to(self.device)

        # Get test data loader
        _, test_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Testing
        logger.info('Testing autoencoder...')
        loss_epoch = 0.0
        n_batches = 0
        start_time = time.time()
        idx_label_score = []
        ae_net.eval()
        with torch.no_grad():
            for data in tqdm(test_loader):
                inputs, labels, idx = data
                inputs = inputs.to(self.device)
                outputs = ae_net(inputs)
                scores = self.test_scores(inputs, outputs)
                loss = torch.mean(scores)

                # Save triple of (idx, label, score) in a list
                idx_label_score += list(zip(idx.cpu().data.numpy().tolist(),
                                            labels.cpu().data.numpy().tolist(),
                                            scores.cpu().data.numpy().tolist()))

                loss_epoch += loss.item()
                n_batches += 1

        logger.info('Test set Loss: {:.8f}'.format(loss_epoch / n_batches))

        _, labels, scores = zip(*idx_label_score)
        labels = np.array(labels)
        scores = np.array(scores)

        auc = roc_auc_score(labels, scores)
        logger.info('Test set AUC: {:.2f}%'.format(100. * auc))

        test_time = time.time() - start_time
        logger.info('Autoencoder testing time: %.3f' % test_time)
        logger.info('Finished testing autoencoder.')

### Contents of `deepSVDD_trainer.py`

In [22]:
class DeepSVDDTrainer(BaseTrainer):
    def __init__(self,
                 R,
                 c,
                 nu: float = 0.1,
                 alpha: float = 50,
                 optimizer_name: str = 'adam',
                 lr: float = 0.001,
                 n_epochs: int = 150,
                 lr_milestones: tuple = (),
                 batch_size: int = 128,
                 weight_decay: float = 1e-6,
                 device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        super().__init__(optimizer_name, lr, n_epochs, lr_milestones, batch_size, weight_decay, device,
                         n_jobs_dataloader)

        # Deep SVDD parameters
        self.R = torch.tensor(R, device=self.device)  # radius R initialized with 0 by default.
        self.c = torch.tensor(c, device=self.device) if c is not None else None
        self.nu = nu
        self.alpha = alpha

        # Results
        self.train_time = None
        self.test_auc = None
        self.test_time = None
        self.test_scores = None

    def objective(self, inputs, reconstructions, latent_features, kl_divergences):
        # Reconstruction loss for Variational Autoencoder (TODO: try with logits)
        # print(f"inputs min: {torch.min(inputs)}, intputs max: {torch.max(inputs)}, outputs min: {torch.min(reconstructions)}, outputs max: {torch.max(reconstructions)}")
        reconstruction_loss = F.binary_cross_entropy(reconstructions, inputs, reduction="none")
        # reconstruction_loss = F.mse_loss(inputs, reconstructions)
        reconstruction_loss = torch.mean(reconstruction_loss.sum(dim=(1, 2, 3)))
        # reconstruction_loss = MSELoss(inputs, reconstructions)

        # Evidence variational lower bound (ELBO)
        kl_div_loss = torch.mean(kl_divergences)
        vae_loss = reconstruction_loss + kl_div_loss

        # Deep SVDD loss
        distances = torch.sum((latent_features - self.c) ** 2, dim=1)
        svdd_loss = torch.mean(distances)

        # Objective function
        loss = svdd_loss + self.alpha * vae_loss
        # print(f"Losses: vae_loss: {vae_loss}, distances: {svdd_loss}, kl_div: {kl_div_loss}, rec_loss: {reconstruction_loss}")
        result = {
            "loss": loss.item(),
            "elbo": vae_loss.item(),
            "reconstruction_loss": reconstruction_loss.item(),
            "kl_divergence_loss": kl_div_loss.item(),
            "svdd_loss": svdd_loss.item()
        }
        return loss, result

    def test_score(self, x, latent_features):
        score = torch.sum((latent_features - self.c) ** 2, dim=1)
        return score

    def train(self, dataset: BaseADDataset, net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        net = net.to(self.device)

        # Get train data loader
        train_loader, _ = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Set optimizer (Adam optimizer for now or RMSprop as it's stated in the paper)
        if self.optimizer_name == "rmsprop":
            optimizer = optim.RMSprop(net.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        else:
            optimizer = optim.Adam(net.parameters(), lr=self.lr, weight_decay=self.weight_decay,
                               amsgrad=self.optimizer_name == 'amsgrad')

        # Set learning rate scheduler
        scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda _: 0.9977)

        # Initialize hypersphere center c (if c not loaded)
        if self.c is None:
            logger.info('Initializing center c...')
            self.c = self.init_center_c(train_loader, net)
            logger.info('Center c initialized.')

        # Training
        logger.info('Starting training...')
        start_time = time.time()
        net.train()
        for epoch in trange(self.n_epochs, desc="Epoch"):
            loss_epoch = 0.0
            all_losses_epoch = {"loss": 0, "elbo": 0, "reconstruction_loss": 0, "kl_divergence_loss": 0, "svdd_loss": 0}
            n_batches = 0
            epoch_start_time = time.time()
            for data in tqdm(train_loader):
                inputs, _, _ = data
                inputs = inputs.to(self.device)

                # Zero the network parameter gradients
                optimizer.zero_grad()

                # Update network parameters via backpropagation: forward + backward + optimize
                reconstructions, latent_features, kl_divs = net(inputs)

                # TODO: test with this on
                # self.c = self.recompute_center_c(latent_features)

                loss, all_losses = self.objective(inputs, reconstructions, latent_features, kl_divs)
                loss.backward()
                optimizer.step()

                loss_epoch += loss.item()
                for key, val in all_losses.items():
                  all_losses_epoch[key] += val
                n_batches += 1

            # Scheduler step
            scheduler.step()
            if epoch in self.lr_milestones:
                logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                        .format(epoch + 1, self.n_epochs, epoch_train_time, loss_epoch / n_batches))
            logger.info(f"All losses: { {key:val / n_batches for key, val in all_losses_epoch.items()} }")

        self.train_time = time.time() - start_time
        logger.info('Training time: %.3f' % self.train_time)
        logger.info('Finished training.')

        # Updating radius
        self.R = self.update_radius(train_loader, net)
        print("New radius after training: ", self.R)

        return net

    def test(self, dataset: BaseADDataset, net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        net = net.to(self.device)

        # Get test data loader
        _, test_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Testing
        logger.info('Starting testing...')
        start_time = time.time()
        idx_label_score = []
        net.eval()
        with torch.no_grad():
            for data in tqdm(test_loader):
                inputs, labels, idx = data
                inputs = inputs.to(self.device)
                _, latent_features, _ = net(inputs)
                scores = self.test_score(inputs, latent_features)

                # Get radius scores (predicted labels)
                predicted_labels = scores.cpu().data.numpy() > self.R
                predicted_labels = predicted_labels * 1
                predicted_labels = predicted_labels.tolist()

                # Save tuples of (idx, label, score, radius_score) in a list
                idx_label_score += list(zip(idx.cpu().data.numpy().tolist(),
                                            labels.cpu().data.numpy().tolist(),
                                            scores.cpu().data.numpy().tolist(),
                                            predicted_labels))

        self.test_time = time.time() - start_time
        logger.info('Testing time: %.3f' % self.test_time)

        self.test_scores = idx_label_score

        # Compute AUC
        _, labels, scores, predicted_labels = zip(*idx_label_score)
        labels = np.array(labels)
        scores = np.array(scores)
        predicted_labels = np.array(predicted_labels)

        self.test_auc = roc_auc_score(labels, scores)
        self.test_auc_preds = roc_auc_score(labels, predicted_labels)
        logger.info('Test set AUC: {:.2f}%'.format(100. * self.test_auc))
        logger.info('Test set predicitons AUC: {:.2f}%'.format(100. * self.test_auc_preds))

        logger.info('Finished testing.')

    def init_center_c(self, train_loader: DataLoader, net: BaseNet, eps=0.1):
        """Initialize hypersphere center c as the mean from an initial forward pass on the data."""
        n_samples = 0
        c = torch.zeros(net.rep_dim, device=self.device)

        net.eval()
        with torch.no_grad():
            for data in tqdm(train_loader):
                # get the inputs of the batch
                inputs, _, _ = data
                inputs = inputs.to(self.device)
                _, outputs, _ = net(inputs)
                n_samples += outputs.shape[0]
                c += torch.sum(outputs, dim=0)

        c /= n_samples

        # If c_i is too close to 0, set to +-eps. Reason: a zero unit can be trivially matched with zero weights.
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps

        return c

    def recompute_center_c(self, latent_features, eps=0.1):
        """Recomputes center c as the mean of latent features"""
        if len(latent_features.shape) == 1:
            return latent_features

        c = torch.mean(latent_features, dim=0)

        # If c_i it too close to 0, set to +- eps. Reason: a zero unit can be trivially matched with zero weights.
        # Note: in VAE this might be obsolete
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps

        return c

    def update_radius(self, train_loader: DataLoader, net: BaseNet):
        "Update Radius after training"
        logger = logging.getLogger()

        logger.info('Updating radius...')
        start_time = time.time()
        distances = []
        net.eval()
        with torch.no_grad():
            for data in tqdm(train_loader):
                inputs, labels, idx = data
                inputs = inputs.to(self.device)
                _, latent_features, _ = net(inputs)
                scores = self.test_score(inputs, latent_features)
                distances.append(scores)
        distances = torch.cat(distances, dim=0)

        new_radius = get_radius(distances, self.nu)
        return new_radius


def get_radius(dist: torch.Tensor, nu: float):
    """Optimally solve for radius R via the (1-nu)-quantile of distances."""
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)

## Main content

### Contents of `deepSVDD.py` (Deep SVDD Model)


In [23]:
class DeepSVDD(object):
    """A class for the Deep SVDD method.

    Attributes:
        objective: A string specifying the Deep SVDD objective (either 'one-class' or 'soft-boundary').
        nu: Deep SVDD hyperparameter nu (must be 0 < nu <= 1).
        R: Hypersphere radius R.
        c: Hypersphere center c.
        net_name: A string indicating the name of the neural network to use.
        net: The neural network \phi.
        ae_net: The autoencoder network corresponding to \phi for network weights pretraining.
        trainer: DeepSVDDTrainer to train a Deep SVDD model.
        optimizer_name: A string indicating the optimizer to use for training the Deep SVDD network.
        ae_trainer: AETrainer to train an autoencoder in pretraining.
        ae_optimizer_name: A string indicating the optimizer to use for pretraining the autoencoder.
        results: A dictionary to save the results.
    """

    def __init__(self, nu: float = 0.1, alpha: float = 5):
        """Inits DeepSVDD with one of the two objectives and hyperparameter nu."""

        assert (0 < nu) & (nu <= 1), "For hyperparameter nu, it must hold: 0 < nu <= 1."
        self.nu = nu
        self.alpha = alpha
        self.R = 0.0  # hypersphere radius R
        self.c = None  # hypersphere center c

        self.net_name = None
        self.net = None  # neural network \phi

        self.trainer = None
        self.optimizer_name = None

        self.ae_net = None  # autoencoder network for pretraining
        self.ae_trainer = None
        self.ae_optimizer_name = None

        self.results = {
            'train_time': None,
            'test_auc': None,
            'test_time': None,
            'test_scores': None,
        }
        print("Deep SVDD initialized")

    def set_network(self, net_name):
        """Builds the neural network \phi."""
        self.net_name = net_name
        self.net = build_network(net_name)

    def train(self,
              dataset: BaseADDataset,
              optimizer_name: str = 'adam',
              lr: float = 0.001,
              n_epochs: int = 50,
              lr_milestones: tuple = (),
              batch_size: int = 128,
              weight_decay: float = 1e-6,
              device: str = 'cuda',
              n_jobs_dataloader: int = 0):
        """Trains the Deep SVDD model on the training data."""

        self.optimizer_name = optimizer_name
        self.trainer = DeepSVDDTrainer(self.R, self.c, self.nu, self.alpha, optimizer_name, lr=lr,
                                       n_epochs=n_epochs, lr_milestones=lr_milestones, batch_size=batch_size,
                                       weight_decay=weight_decay, device=device, n_jobs_dataloader=n_jobs_dataloader)
        print("DeepSVDDTrainer initialized")
        # Get the model
        self.net = self.trainer.train(dataset, self.net)
        self.R = float(self.trainer.R)
        self.c = self.trainer.c.cpu().data.numpy().tolist()  # get list
        self.results['train_time'] = self.trainer.train_time

    def test(self, dataset: BaseADDataset, device: str = 'cuda', n_jobs_dataloader: int = 0):
        """Tests the Deep SVDD model on the test data."""

        if self.trainer is None:
            self.trainer = DeepSVDDTrainer(self.objective, self.R, self.c, self.nu,
                                           device=device, n_jobs_dataloader=n_jobs_dataloader)

        self.trainer.test(dataset, self.net)
        # Get results
        self.results['test_auc'] = self.trainer.test_auc
        self.results['test_time'] = self.trainer.test_time
        self.results['test_scores'] = self.trainer.test_scores

    def pretrain(self,
                 dataset: BaseADDataset,
                 optimizer_name: str = 'adam',
                 lr: float = 0.001,
                 n_epochs: int = 100,
                 lr_milestones: tuple = (),
                 batch_size: int = 128,
                 weight_decay: float = 1e-6,
                 device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        """Pretrains the weights for the Deep SVDD network \phi via autoencoder."""

        self.ae_net = build_autoencoder(self.net_name)
        self.ae_optimizer_name = optimizer_name
        self.ae_trainer = AETrainer(optimizer_name, lr=lr, n_epochs=n_epochs, lr_milestones=lr_milestones,
                                    batch_size=batch_size, weight_decay=weight_decay, device=device,
                                    n_jobs_dataloader=n_jobs_dataloader)
        self.ae_net = self.ae_trainer.train(dataset, self.ae_net)
        self.ae_trainer.test(dataset, self.ae_net)
        self.init_network_weights_from_pretraining()

    def init_network_weights_from_pretraining(self):
        """Initialize the Deep SVDD network weights from the encoder weights of the pretraining autoencoder."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()

        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)
        # Load the new state_dict
        self.net.load_state_dict(net_dict)

    def save_model(self, export_model, save_ae=True):
        """Save Deep SVDD model to export_model."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict() if save_ae else None

        torch.save({'R': self.R,
                    'c': self.c,
                    'net_dict': net_dict,
                    'ae_net_dict': ae_net_dict}, export_model)

    def load_model(self, model_path, load_ae=False):
        """Load Deep SVDD model from model_path."""

        model_dict = torch.load(model_path)

        self.R = model_dict['R']
        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])
        if load_ae:
            if self.ae_net is None:
                self.ae_net = build_autoencoder(self.net_name)
            self.ae_net.load_state_dict(model_dict['ae_net_dict'])

    def save_results(self, export_json):
        """Save results dict to a JSON-file."""
        with open(export_json, 'w') as fp:
            json.dump(self.results, fp)


### Contents of `main.py`

In [24]:
################################################################################
# Settings
################################################################################
# @click.command()
# @click.argument('dataset_name', type=click.Choice(['mnist', 'cifar10']))
# @click.argument('net_name', type=click.Choice(['mnist_LeNet', 'cifar10_LeNet', 'cifar10_LeNet_ELU']))
# @click.argument('xp_path', type=click.Path(exists=True))
# @click.argument('data_path', type=click.Path(exists=True))
# @click.option('--load_config', type=click.Path(exists=True), default=None,
#               help='Config JSON-file path (default: None).')
# @click.option('--load_model', type=click.Path(exists=True), default=None,
#               help='Model file path (default: None).')
# @click.option('--objective', type=click.Choice(['one-class', 'soft-boundary']), default='one-class',
#               help='Specify Deep SVDD objective ("one-class" or "soft-boundary").')
# @click.option('--nu', type=float, default=0.1, help='Deep SVDD hyperparameter nu (must be 0 < nu <= 1).')
# @click.option('--device', type=str, default='cuda', help='Computation device to use ("cpu", "cuda", "cuda:2", etc.).')
# @click.option('--seed', type=int, default=-1, help='Set seed. If -1, use randomization.')
# @click.option('--optimizer_name', type=click.Choice(['adam', 'amsgrad']), default='adam',
#               help='Name of the optimizer to use for Deep SVDD network training.')
# @click.option('--lr', type=float, default=0.001,
#               help='Initial learning rate for Deep SVDD network training. Default=0.001')
# @click.option('--n_epochs', type=int, default=50, help='Number of epochs to train.')
# @click.option('--lr_milestone', type=int, default=0, multiple=True,
#               help='Lr scheduler milestones at which lr is multiplied by 0.1. Can be multiple and must be increasing.')
# @click.option('--batch_size', type=int, default=128, help='Batch size for mini-batch training.')
# @click.option('--weight_decay', type=float, default=1e-6,
#               help='Weight decay (L2 penalty) hyperparameter for Deep SVDD objective.')
# @click.option('--pretrain', type=bool, default=True,
#               help='Pretrain neural network parameters via autoencoder.')
# @click.option('--ae_optimizer_name', type=click.Choice(['adam', 'amsgrad']), default='adam',
#               help='Name of the optimizer to use for autoencoder pretraining.')
# @click.option('--ae_lr', type=float, default=0.001,
#               help='Initial learning rate for autoencoder pretraining. Default=0.001')
# @click.option('--ae_n_epochs', type=int, default=100, help='Number of epochs to train autoencoder.')
# @click.option('--ae_lr_milestone', type=int, default=0, multiple=True,
#               help='Lr scheduler milestones at which lr is multiplied by 0.1. Can be multiple and must be increasing.')
# @click.option('--ae_batch_size', type=int, default=128, help='Batch size for mini-batch autoencoder training.')
# @click.option('--ae_weight_decay', type=float, default=1e-6,
#               help='Weight decay (L2 penalty) hyperparameter for autoencoder objective.')
# @click.option('--n_jobs_dataloader', type=int, default=0,
#               help='Number of workers for data loading. 0 means that the data will be loaded in the main process.')
# @click.option('--normal_class', type=int, default=0,
#               help='Specify the normal class of the dataset (all other classes are considered anomalous).')
# def main(dataset_name, net_name, xp_path, data_path, load_config, load_model, objective, nu, device, seed,
#          optimizer_name, lr, n_epochs, lr_milestone, batch_size, weight_decay, pretrain, ae_optimizer_name, ae_lr,
#          ae_n_epochs, ae_lr_milestone, ae_batch_size, ae_weight_decay, n_jobs_dataloader, normal_class):
#     """
#     Deep SVDD, a fully deep method for anomaly detection.

#     :arg DATASET_NAME: Name of the dataset to load.
#     :arg NET_NAME: Name of the neural network to use.
#     :arg XP_PATH: Export path for logging the experiment.
#     :arg DATA_PATH: Root path of data.
#     """
#     return run(dataset_name, net_name, xp_path, data_path, load_config, load_model, objective, nu, device, seed,
#          optimizer_name, lr, n_epochs, lr_milestone, batch_size, weight_decay, pretrain, ae_optimizer_name, ae_lr,
#          ae_n_epochs, ae_lr_milestone, ae_batch_size, ae_weight_decay, n_jobs_dataloader, normal_class)

In [25]:
def run(dataset_name,
        net_name,
        xp_path,
        data_path,
        load_config,
        load_model,
        nu,
        alpha,
        device,
        seed,
        optimizer_name,
        lr,
        n_epochs,
        lr_milestone,
        batch_size,
        weight_decay,
        pretrain,
        ae_optimizer_name,
        ae_lr,
        ae_n_epochs,
        ae_lr_milestone,
        ae_batch_size,
        ae_weight_decay,
        n_jobs_dataloader,
        normal_class):
    """
    Deep SVDD, a fully deep method for anomaly detection.

    :arg DATASET_NAME: Name of the dataset to load.
    :arg NET_NAME: Name of the neural network to use.
    :arg XP_PATH: Export path for logging the experiment.
    :arg DATA_PATH: Root path of data.
    """

    # Prehandle parameters
    if type(lr_milestone) == int:
      lr_milestone = [lr_milestone]
    if type(ae_lr_milestone) == int:
      ae_lr_milestone = [ae_lr_milestone]

    # Get configuration
    cfg = Config(locals().copy())

    # Set up logging
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    log_file = xp_path + '/log.txt'
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # Print arguments
    logger.info('Log file is %s.' % log_file)
    logger.info('Data path is %s.' % data_path)
    logger.info('Export path is %s.' % xp_path)

    logger.info('Dataset: %s' % dataset_name)
    logger.info('Normal class: %d' % normal_class)
    logger.info('Network: %s' % net_name)

    # If specified, load experiment config from JSON-file
    if load_config:
        cfg.load_config(import_json=load_config)
        logger.info('Loaded configuration from %s.' % load_config)

    # Print configuration
    # logger.info('Deep SVDD objective: %s' % cfg.settings['objective'])
    # logger.info('Nu-paramerter: %.2f' % cfg.settings['nu'])

    # Set seed
    if cfg.settings['seed'] != -1:
        random.seed(cfg.settings['seed'])
        np.random.seed(cfg.settings['seed'])
        torch.manual_seed(cfg.settings['seed'])
        logger.info('Set seed to %d.' % cfg.settings['seed'])

    # Default device to 'cpu' if cuda is not available
    if not torch.cuda.is_available():
        device = 'cpu'
    logger.info('Computation device: %s' % device)
    logger.info('Number of dataloader workers: %d' % n_jobs_dataloader)

    # Load data
    dataset = load_dataset(dataset_name, data_path, normal_class)

    # Initialize DeepSVDD model and set neural network \phi
    deep_SVDD = DeepSVDD(cfg.settings['nu'], cfg.settings['alpha'])
    deep_SVDD.set_network(net_name)
    # If specified, load Deep SVDD model (radius R, center c, network weights, and possibly autoencoder weights)
    if load_model:
        deep_SVDD.load_model(model_path=load_model, load_ae=True)
        logger.info('Loading model from %s.' % load_model)

    logger.info('Pretraining: %s' % pretrain)
    if pretrain:
        # Log pretraining details
        logger.info('Pretraining optimizer: %s' % cfg.settings['ae_optimizer_name'])
        logger.info('Pretraining learning rate: %g' % cfg.settings['ae_lr'])
        logger.info('Pretraining epochs: %d' % cfg.settings['ae_n_epochs'])
        logger.info('Pretraining learning rate scheduler milestones: %s' % (cfg.settings['ae_lr_milestone'],))
        logger.info('Pretraining batch size: %d' % cfg.settings['ae_batch_size'])
        logger.info('Pretraining weight decay: %g' % cfg.settings['ae_weight_decay'])

        # Pretrain model on dataset (via autoencoder)
        deep_SVDD.pretrain(dataset,
                           optimizer_name=cfg.settings['ae_optimizer_name'],
                           lr=cfg.settings['ae_lr'],
                           n_epochs=cfg.settings['ae_n_epochs'],
                           lr_milestones=cfg.settings['ae_lr_milestone'],
                           batch_size=cfg.settings['ae_batch_size'],
                           weight_decay=cfg.settings['ae_weight_decay'],
                           device=device,
                           n_jobs_dataloader=n_jobs_dataloader)

    # Log training details
    logger.info('Training optimizer: %s' % cfg.settings['optimizer_name'])
    logger.info('Training learning rate: %g' % cfg.settings['lr'])
    logger.info('Training epochs: %d' % cfg.settings['n_epochs'])
    logger.info('Training learning rate scheduler milestones: %s' % (cfg.settings['lr_milestone'],))
    logger.info('Training batch size: %d' % cfg.settings['batch_size'])
    logger.info('Training weight decay: %g' % cfg.settings['weight_decay'])

    # Train model on dataset
    logger.debug(deep_SVDD)
    deep_SVDD.train(dataset,
                    optimizer_name=cfg.settings['optimizer_name'],
                    lr=cfg.settings['lr'],
                    n_epochs=cfg.settings['n_epochs'],
                    lr_milestones=cfg.settings['lr_milestone'],
                    batch_size=cfg.settings['batch_size'],
                    weight_decay=cfg.settings['weight_decay'],
                    device=device,
                    n_jobs_dataloader=n_jobs_dataloader)

    # Test model
    deep_SVDD.test(dataset, device=device, n_jobs_dataloader=n_jobs_dataloader)

    # Plot most anomalous and most normal (within-class) test samples
    indices, labels, scores, _ = zip(*deep_SVDD.results['test_scores'])
    indices, labels, scores = np.array(indices), np.array(labels), np.array(scores)
    idx_sorted = indices[labels == 0][np.argsort(scores[labels == 0])]  # sorted from lowest to highest anomaly score

    if dataset_name in ('mnist', 'cifar10'):

        if dataset_name == 'mnist':
            X_normals = dataset.test_set.test_data[idx_sorted[:32], ...].unsqueeze(1)
            X_outliers = dataset.test_set.test_data[idx_sorted[-32:], ...].unsqueeze(1)

        if dataset_name == 'cifar10':
            X_normals = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[:32], ...], (0, 3, 1, 2)))
            X_outliers = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[-32:], ...], (0, 3, 1, 2)))

        plot_images_grid(X_normals, export_img=xp_path + '/normals', title='Most normal examples', padding=2)
        plot_images_grid(X_outliers, export_img=xp_path + '/outliers', title='Most anomalous examples', padding=2)

    # Save results, model, and configuration
    deep_SVDD.save_results(export_json=xp_path + '/results.json')
    deep_SVDD.save_model(export_model=xp_path + '/model.tar', save_ae=pretrain)
    cfg.save_config(export_json=xp_path + '/config.json')

    return deep_SVDD

## Running SVDD training and results

In [26]:
!mkdir results

In [ ]:
model = run(
    dataset_name="mnist",
    net_name="mnist_LeNet",
    xp_path="./results",
    data_path="./",
    load_config=None,
    load_model=None,
    nu=0.05,
    alpha=1,
    device="cuda",
    seed=-1,
    optimizer_name="adam",
    lr=5e-4,
    n_epochs=150,
    lr_milestone=[150, 200],
    batch_size=200,
    weight_decay=1e-6,
    pretrain=False,
    ae_optimizer_name="adam",
    ae_lr=0.001,
    ae_n_epochs=100,
    ae_lr_milestone=0,
    ae_batch_size=32,
    ae_weight_decay=1e-6,
    n_jobs_dataloader=0,
    normal_class=0
)

# @click.argument('dataset_name', type=click.Choice(['mnist', 'cifar10']))
# @click.argument('net_name', type=click.Choice(['mnist_LeNet', 'cifar10_LeNet', 'cifar10_LeNet_ELU']))
# @click.argument('xp_path', type=click.Path(exists=True))
# @click.argument('data_path', type=click.Path(exists=True))
# @click.option('--load_config', type=click.Path(exists=True), default=None,
#               help='Config JSON-file path (default: None).')
# @click.option('--load_model', type=click.Path(exists=True), default=None,
#               help='Model file path (default: None).')
# @click.option('--objective', type=click.Choice(['one-class', 'soft-boundary']), default='one-class',
#               help='Specify Deep SVDD objective ("one-class" or "soft-boundary").')
# @click.option('--nu', type=float, default=0.1, help='Deep SVDD hyperparameter nu (must be 0 < nu <= 1).')
# @click.option('--device', type=str, default='cuda', help='Computation device to use ("cpu", "cuda", "cuda:2", etc.).')
# @click.option('--seed', type=int, default=-1, help='Set seed. If -1, use randomization.')
# @click.option('--optimizer_name', type=click.Choice(['adam', 'amsgrad']), default='adam',
#               help='Name of the optimizer to use for Deep SVDD network training.')
# @click.option('--lr', type=float, default=0.001,
#               help='Initial learning rate for Deep SVDD network training. Default=0.001')
# @click.option('--n_epochs', type=int, default=50, help='Number of epochs to train.')
# @click.option('--lr_milestone', type=int, default=0, multiple=True,
#               help='Lr scheduler milestones at which lr is multiplied by 0.1. Can be multiple and must be increasing.')
# @click.option('--batch_size', type=int, default=128, help='Batch size for mini-batch training.')
# @click.option('--weight_decay', type=float, default=1e-6,
#               help='Weight decay (L2 penalty) hyperparameter for Deep SVDD objective.')
# @click.option('--pretrain', type=bool, default=True,
#               help='Pretrain neural network parameters via autoencoder.')
# @click.option('--ae_optimizer_name', type=click.Choice(['adam', 'amsgrad']), default='adam',
#               help='Name of the optimizer to use for autoencoder pretraining.')
# @click.option('--ae_lr', type=float, default=0.001,
#               help='Initial learning rate for autoencoder pretraining. Default=0.001')
# @click.option('--ae_n_epochs', type=int, default=100, help='Number of epochs to train autoencoder.')
# @click.option('--ae_lr_milestone', type=int, default=0, multiple=True,
#               help='Lr scheduler milestones at which lr is multiplied by 0.1. Can be multiple and must be increasing.')
# @click.option('--ae_batch_size', type=int, default=128, help='Batch size for mini-batch autoencoder training.')
# @click.option('--ae_weight_decay', type=float, default=1e-6,
#               help='Weight decay (L2 penalty) hyperparameter for autoencoder objective.')
# @click.option('--n_jobs_dataloader', type=int, default=0,
#               help='Number of workers for data loading. 0 means that the data will be loaded in the main process.')
# @click.option('--normal_class', type=int, default=0,
#               help='Specify the normal class of the dataset (all other classes are considered anomalous).')


In [54]:
logger = logging.getLogger()

dataset = load_dataset("mnist", "./", 0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get test data loader
_, test_loader = dataset.loaders(batch_size=200, num_workers=0)

# Testing
net = model.net
R = model.R
print(f"R = {R}")

logger.info('Starting testing...')
start_time = time.time()
idx_label_score = []
net.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        inputs, labels, idx = data
        inputs = inputs.to(device)
        _, latent_features, _ = net(inputs)
        scores = model.trainer.test_score(inputs, latent_features)

        # Get radius scores (predicted labels)
        predicted_labels = scores.cpu().data.numpy() > R
        predicted_labels = predicted_labels * 1
        predicted_labels = predicted_labels.tolist()

        # Save tuples of (idx, label, score, radius_score) in a list
        idx_label_score += list(zip(idx.cpu().data.numpy().tolist(),
                                    labels.cpu().data.numpy().tolist(),
                                    scores.cpu().data.numpy().tolist(),
                                    predicted_labels))

test_time = time.time() - start_time
logger.info('Testing time: %.3f' % test_time)

test_scores = idx_label_score

# Compute AUC
_, labels, scores, predicted_labels = zip(*idx_label_score)
labels = np.array(labels)
scores = np.array(scores)
predicted_labels = np.array(predicted_labels)

test_auc = roc_auc_score(labels, scores)
test_auc_preds = roc_auc_score(labels, predicted_labels)
logger.info('Test set AUC: {:.2f}%'.format(100. * test_auc))
logger.info('Test set predicitons AUC: {:.2f}%'.format(100. * test_auc_preds))

logger.info('Finished testing.')


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
INFO:root:Starting testing...


R = 3.926437735557556


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
INFO:root:Testing time: 3.827
INFO:root:Test set AUC: 64.10%
INFO:root:Test set predicitons AUC: 50.05%
INFO:root:Finished testing.


In [55]:
test = {}
res = {}
for lb in labels:
    test[lb] = test.get(lb, 0) + 1

for lb in predicted_labels:
    res[lb] = res.get(lb, 0) + 1

print(test, res)

{1: 9020, 0: 980} {1: 9998, 0: 2}


In [56]:
scores.min()

3.73774790763855

# VAE FROM GIT

## Addons

In [47]:
# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
log_file = './results/log.txt'
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable


# Acknowledgements: https://github.com/wohlert/semi-supervised-pytorch
class Stochastic(nn.Module):
    """
    Base stochastic layer that uses the reparametrization trick (Kingma and Welling, 2013) to draw a sample from a
    distribution parametrized by mu and log_var.
    """

    def __init__(self):
        super(Stochastic, self).__init__()

    def reparametrize(self, mu, log_var):
        epsilon = Variable(torch.randn(mu.size()), requires_grad=False)

        if mu.is_cuda:
            epsilon = epsilon.to(mu.device)

        # log_std = 0.5 * log_var
        # std = exp(log_std)
        std = log_var.mul(0.5).exp_()

        # z = std * epsilon + mu
        z = mu.addcmul(std, epsilon)

        return z

    def forward(self, x):
        raise NotImplementedError


class GaussianSample(Stochastic):
    """
    Layer that represents a sample from a Gaussian distribution.
    """

    def __init__(self, in_features, out_features):
        super(GaussianSample, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.mu = nn.Linear(in_features, out_features)
        self.log_var = nn.Linear(in_features, out_features)

    def forward(self, x):
        mu = self.mu(x)
        log_var = F.softplus(self.log_var(x))
        return self.reparametrize(mu, log_var), mu, log_var

In [28]:
import math
import torch
import torch.nn.functional as F


# Acknowledgements: https://github.com/wohlert/semi-supervised-pytorch
def log_standard_gaussian(x):
    """
    Evaluates the log pdf of a standard normal distribution at x.

    :param x: point to evaluate
    :return: log N(x|0,I)
    """
    return torch.sum(-0.5 * math.log(2 * math.pi) - x ** 2 / 2, dim=-1)


def log_gaussian(x, mu, log_var):
    """
    Evaluates the log pdf of a normal distribution parametrized by mu and log_var at x.

    :param x: point to evaluate
    :param mu: mean
    :param log_var: log variance
    :return: log N(x|µ,σI)
    """
    log_pdf = -0.5 * math.log(2 * math.pi) - log_var / 2 - (x - mu)**2 / (2 * torch.exp(log_var))
    return torch.sum(log_pdf, dim=-1)


def log_standard_categorical(p):
    """
    Computes the cross-entropy between a (one-hot) categorical vector and a standard (uniform) categorical distribution.
    :param p: one-hot categorical distribution
    :return: H(p,u)
    """
    eps = 1e-8
    prior = F.softmax(torch.ones_like(p), dim=1)  # Uniform prior over y
    prior.requires_grad = False
    cross_entropy = -torch.sum(p * torch.log(prior + eps), dim=1)

    return cross_entropy

## Vartiational Autoencoder

In [29]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init


# Acknowledgements: https://github.com/wohlert/semi-supervised-pytorch
class Encoder(nn.Module):
    """
    Encoder, i.e. the inference network.

    Attempts to infer the latent probability distribution p(z|x) from the data x by fitting a
    variational distribution q_φ(z|x). Returns the two parameters of the distribution (µ, log σ²).

    :param dims: dimensions of the network given by [input_dim, [hidden_dims], latent_dim].
    """

    def __init__(self, dims, sample_layer=GaussianSample):
        super(Encoder, self).__init__()

        [x_dim, h_dim, z_dim] = dims
        neurons = [x_dim, *h_dim]
        linear_layers = [nn.Linear(neurons[i-1], neurons[i]) for i in range(1, len(neurons))]

        self.hidden = nn.ModuleList(linear_layers)
        self.sample = sample_layer(h_dim[-1], z_dim)

    def forward(self, x):
        for layer in self.hidden:
            x = F.relu(layer(x))
        return self.sample(x)


class Decoder(nn.Module):
    """
    Decoder, i.e. the generative network.

    Generates samples from an approximation p_θ(x|z) of the original distribution p(x)
    by transforming a latent representation z.

    :param dims: dimensions of the network given by [latent_dim, [hidden_dims], input_dim].
    """

    def __init__(self, dims):
        super(Decoder, self).__init__()

        [z_dim, h_dim, x_dim] = dims
        neurons = [z_dim, *h_dim]
        linear_layers = [nn.Linear(neurons[i-1], neurons[i]) for i in range(1, len(neurons))]

        self.hidden = nn.ModuleList(linear_layers)
        self.reconstruction = nn.Linear(h_dim[-1], x_dim)
        self.output_activation = nn.Sigmoid()

    def forward(self, x):
        for layer in self.hidden:
            x = F.relu(layer(x))
        return self.output_activation(self.reconstruction(x))


class VariationalAutoencoder(nn.Module):
    """
    Variational Autoencoder (VAE) (Kingma and Welling, 2013) model consisting of an encoder-decoder pair for which
    a variational distribution is fitted to the encoder.
    Also known as the M1 model in (Kingma et al., 2014)

    :param  dims: dimensions of the networks given by [input_dim, latent_dim, [hidden_dims]]. Encoder and decoder
    are build symmetrically.
    """

    def __init__(self, dims):
        super(VariationalAutoencoder, self).__init__()

        [x_dim, z_dim, h_dim] = dims
        self.z_dim = z_dim
        self.flow = None

        self.encoder = Encoder([x_dim, h_dim, z_dim])
        self.decoder = Decoder([z_dim, list(reversed(h_dim)), x_dim])
        self.kl_divergence = 0

        # Init linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

    def _kld(self, z, q_param, p_param=None):
        """
        Computes the KL-divergence of some latent variable z.

        KL(q||p) = - ∫ q(z) log [ p(z) / q(z) ] = - E_q[ log p(z) - log q(z) ]

        :param z: sample from q-distribuion
        :param q_param: (mu, log_var) of the q-distribution
        :param p_param: (mu, log_var) of the p-distribution
        :return: KL(q||p)
        """
        (mu, log_var) = q_param

        if self.flow is not None:
            f_z, log_det_z = self.flow(z)
            qz = log_gaussian(z, mu, log_var) - sum(log_det_z)
            z = f_z
        else:
            qz = log_gaussian(z, mu, log_var)

        if p_param is None:
            pz = log_standard_gaussian(z)
        else:
            (mu, log_var) = p_param
            pz = log_gaussian(z, mu, log_var)

        kl = qz - pz

        return kl

    def add_flow(self, flow):
        self.flow = flow

    def forward(self, x, y=None):
        """
        Runs a forward pass on a data point through the VAE model to provide its reconstruction and the parameters of
        the variational approximate distribution q.

        :param x: input data
        :return: reconstructed input
        """
        z, q_mu, q_log_var = self.encoder(x)
        self.kl_divergence = self._kld(z, (q_mu, q_log_var))
        rec = self.decoder(z)

        return rec

    def sample(self, z):
        """
        Given z ~ N(0, I) generates a sample from the learned distribution based on p_θ(x|z).

        :param z: (torch.autograd.Variable) latent normal variable
        :return: (torch.autograd.Variable) generated sample
        """
        return self.decoder(z)

## Trainer VAE

In [50]:
# from base.base_trainer import BaseTrainer
# from base.base_dataset import BaseADDataset
# from base.base_net import BaseNet
from sklearn.metrics import roc_auc_score

import logging
import time
import torch
import torch.optim as optim
import numpy as np


def binary_cross_entropy(x, y):
    eps = 1e-8
    return -torch.sum(y * torch.log(x + eps) + (1 - y) * torch.log(1 - x + eps), dim=-1)


class VAETrainer(BaseTrainer):

    def __init__(self, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 150, lr_milestones: tuple = (),
                 batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda', n_jobs_dataloader: int = 0):
        super().__init__(optimizer_name, lr, n_epochs, lr_milestones, batch_size, weight_decay, device,
                         n_jobs_dataloader)

        # Results
        self.train_time = None
        self.test_auc = None
        self.test_time = None

    def train(self, dataset: BaseADDataset, vae: BaseNet):
        logger = logging.getLogger()

        # Get train data loader
        train_loader, _ = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Set device
        vae = vae.to(self.device)

        # Set optimizer (Adam optimizer for now)
        optimizer = optim.Adam(vae.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        # Set learning rate scheduler
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_milestones, gamma=0.1)

        # Training
        logger.info('Starting pretraining...')
        start_time = time.time()
        vae.train()
        for epoch in trange(self.n_epochs):

            scheduler.step()
            if epoch in self.lr_milestones:
                logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

            epoch_loss = 0.0
            n_batches = 0
            epoch_start_time = time.time()
            for data in tqdm(train_loader):
                inputs, _, _ = data
                inputs = inputs.to(self.device)
                inputs = inputs.view(inputs.size(0), -1)

                # Zero the network parameter gradients
                optimizer.zero_grad()

                # Update network parameters via backpropagation: forward + backward + optimize
                rec = vae(inputs)

                likelihood = -binary_cross_entropy(rec, inputs)
                elbo = likelihood - vae.kl_divergence

                # Overall loss
                loss = -torch.mean(elbo)

                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                n_batches += 1

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logger.info(f'| Epoch: {epoch + 1:03}/{self.n_epochs:03} | Train Time: {epoch_train_time:.3f}s '
                        f'| Train Loss: {epoch_loss / n_batches:.6f} |')

        self.train_time = time.time() - start_time
        logger.info('Pretraining Time: {:.3f}s'.format(self.train_time))
        logger.info('Finished pretraining.')

        return vae

    def test(self, dataset: BaseADDataset, vae: BaseNet):
        logger = logging.getLogger()

        # Get test data loader
        _, test_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Set device
        vae = vae.to(self.device)

        # Testing
        logger.info('Starting testing...')
        epoch_loss = 0.0
        n_batches = 0
        start_time = time.time()
        idx_label_score = []
        vae.eval()
        with torch.no_grad():
            for data in tqdm(test_loader):
                inputs, labels, idx = data
                inputs, labels, idx = inputs.to(self.device), labels.to(self.device), idx.to(self.device)

                inputs = inputs.view(inputs.size(0), -1)

                rec = vae(inputs)
                likelihood = -binary_cross_entropy(rec, inputs)
                scores = -likelihood  # negative likelihood as anomaly score

                # Save triple of (idx, label, score) in a list
                idx_label_score += list(zip(idx.cpu().data.numpy().tolist(),
                                            labels.cpu().data.numpy().tolist(),
                                            scores.cpu().data.numpy().tolist()))

                # Overall loss
                elbo = likelihood - vae.kl_divergence
                loss = -torch.mean(elbo)

                print(f"Elbo: {-loss.item()}, Likelihood: {likelihood.mean()}, KL: {vae.kl_divergence.mean()}")

                epoch_loss += loss.item()
                n_batches += 1

        self.test_time = time.time() - start_time

        # Compute AUC
        _, labels, scores = zip(*idx_label_score)
        labels = np.array(labels)
        scores = np.array(scores)
        self.test_auc = roc_auc_score(labels, scores)

        # Log results
        logger.info('Test Loss: {:.6f}'.format(epoch_loss / n_batches))
        logger.info('Test AUC: {:.2f}%'.format(100. * self.test_auc))
        logger.info('Test Time: {:.3f}s'.format(self.test_time))
        logger.info('Finished testing variational autoencoder.')

def get_radius(dist: torch.Tensor, nu: float):
    """Optimally solve for radius R via the (1-nu)-quantile of distances."""
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)

In [55]:
vaetrainer = VAETrainer(
    optimizer_name='adam',
    lr=0.0001,
    n_epochs=150,
    lr_milestones=(),
    batch_size=128,
    weight_decay=1e-6,
    device='cuda',
    n_jobs_dataloader=0
)

dataset = load_dataset("mnist", "./", 0)
vaenet = VariationalAutoencoder(dims=[784, 32, [256, 128]])

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [56]:
new_vaenet = vaetrainer.train(dataset, vaenet)

INFO:root:Starting pretraining...


  0%|          | 0/150 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
INFO:root:| Epoch: 001/150 | Train Time: 4.624s | Train Loss: 512.346793 |


...

INFO:root:| Epoch: 148/150 | Train Time: 2.793s | Train Loss: 194.079558 |


  0%|          | 0/47 [00:00<?, ?it/s]

INFO:root:| Epoch: 149/150 | Train Time: 2.813s | Train Loss: 194.090099 |


  0%|          | 0/47 [00:00<?, ?it/s]

INFO:root:| Epoch: 150/150 | Train Time: 2.823s | Train Loss: 194.154412 |
INFO:root:Pretraining Time: 434.529s
INFO:root:Finished pretraining.


In [ ]:
vaetrainer.test(dataset, vaenet)

Same thing :/

# Testing site

In [ ]:
temp = 'dataset_name, net_name, xp_path, data_path, load_config, load_model, objective, nu, device, seed, optimizer_name, lr, n_epochs, lr_milestone, batch_size, weight_decay, pretrain, ae_optimizer_name, ae_lr, ae_n_epochs, ae_lr_milestone, ae_batch_size, ae_weight_decay, n_jobs_dataloader, normal_class'
temp = [f"{tmp}=None" for tmp in temp.split(", ")]
print(",\n".join(temp))

dataset_name=None,
net_name=None,
xp_path=None,
data_path=None,
load_config=None,
load_model=None,
objective=None,
nu=None,
device=None,
seed=None,
optimizer_name=None,
lr=None,
n_epochs=None,
lr_milestone=None,
batch_size=None,
weight_decay=None,
pretrain=None,
ae_optimizer_name=None,
ae_lr=None,
ae_n_epochs=None,
ae_lr_milestone=None,
ae_batch_size=None,
ae_weight_decay=None,
n_jobs_dataloader=None,
normal_class=None


In [ ]:
def lol(kaka=2):
  print(locals())

lol(3)

{'kaka': 3}


In [ ]:
type(2) == int

True

In [ ]:
torch.Tensor([1,2,3]).to(torch.device("cuda:0"))

tensor([1., 2., 3.], device='cuda:0')

In [ ]:
t = torch.randn(2, 4)
t

tensor([[-1.7728,  0.0940, -0.3323,  0.6219],
        [ 0.6259,  1.0272,  1.6726, -1.7340]])

In [ ]:
torch.max(t, torch.zeros_like(t))

tensor([[0.0000, 0.0940, 0.0000, 0.6219],
        [0.6259, 1.0272, 1.6726, 0.0000]])

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi